In [1]:
import copy
import numpy as np
from matplotlib import pyplot as plt
from common.baseclasses import AWA
from importlib import reload
from common import numerical_recipes as numrec
from common import plotting
from common.numerical_recipes import QuickConvolver,smooth
import EigenProbe as EP
from NearFieldOptics import Materials as M

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import warnings
warnings.filterwarnings('ignore')

/Users/alexandersmcleod/tools/python/common/plotting.py:56: MatplotlibDeprecationWarning: Passing raw data via parameters data and lut to register_cmap() is deprecated since 3.3 and will become an error two minor releases later. Instead use: register_cmap(cmap=LinearSegmentedColormap(name, data, lut))
  pyplot.register_cmap(name='BWR', data=cdict)
/Users/alexandersmcleod/tools/python/common/plotting.py:78: MatplotlibDeprecationWarning: Passing raw data via parameters data and lut to register_cmap() is deprecated since 3.3 and will become an error two minor releases later. Instead use: register_cmap(cmap=LinearSegmentedColormap(name, data, lut))
  pyplot.register_cmap(name='BWR2', data=cdict)
/Users/alexandersmcleod/tools/python/common/plotting.py:100: MatplotlibDeprecationWarning: Passing raw data via parameters data and lut to register_cmap() is deprecated since 3.3 and will become an error two minor releases later. Instead use: register_cmap(cmap=LinearSegmentedColormap(name, data, l

<plotting>:
	Registered colormaps "Warm" and "Warm_r"...
<plotting>:
	Registered colormaps "vanheum" and "vanheum_r"...
<plotting>:
	Registered colormaps "NT-MDT" and "NT-MDT_r"...
<plotting>:
	Registered colormaps "rainbow" and "rainbow_r"...
<plotting>:
	Registered colormaps "jingdi" and "jingdi_r"...
<plotting>:
	Registered colormaps "Halcyon" and "Halcyon_r"...
<plotting>:
	Registered colormaps "Cold" and "Cold_r"...
<plotting>:
	Registered colormaps "Sky" and "Sky_r"...
<plotting>:
	Registered colormaps "Gwyddion.net" and "Gwyddion.net_r"...
<plotting>:
	Registered colormaps "BlueRed" and "BlueRed_r"...
<plotting>:
	Registered colormaps "vanheum3" and "vanheum3_r"...
<plotting>:
	Registered colormaps "vanheum2" and "vanheum2_r"...
<material_types.TabulatedMaterialFromFile.__init__>:
	Loading tabulated material data from file "Bi2Se3_epsilon.pickle"...
<material_types.TabulatedMaterialFromFile.__init__>:
	Loading tabulated material data from file "PMMA_epsilon.pickle"...
<material_

# Setup

In [2]:
#--- Imports
from EigenProbe import tip_modeling as TM

#--- Build rectangular laplacian with edge for graphene (particle-in-box planewaves)
N=300
L=200 #We will deal in units of tip radius, so this will be a 6 micron (or so) view
Nq=1000

#--- Dielectric susbstrate with plane wave basis
Substrate=EP.SubstrateDielectric(beta=0.5,Lx=L,Nx=N,Ly=L,Nqmax=Nq,include_const=False)

#--- Build ribbon graphene
Rx=0.6*L
Ry=0.6*L
Graphene=EP.S.SpectralLaplacian_rect(Lx=L,Nx=N,Ly=L,Nqmax=Nq,Rx=Rx,Ry=Ry)

Generating eigenpairs on x,y=[-100.0:+100.0:300],[-100.0:+100.0:300]
	Time elapsed: 1.5918281078338623
Generating eigenpairs on x,y=[-100.0:+100.0:300],[-100.0:+100.0:300]
	Time elapsed: 1.2142281532287598


## Photonic System

In [3]:
Q=20
PS=EP.Photonic2DSystem(Substrate,(Graphene,),
                         beta_substrate=0.5,lambdap=20,sigmas2D=[1-1j/Q],
                       PML=None,
                       PML_amplitude=1,\
                       basis=None)
                         #Nbasis=Nq,qmin=2*np.pi/L)

Constructing augmented basis with 2D materials...
Finding augmented basis by QR decomposition...
	Removed 0 redundant vectors.
	Time elapsed: 21.33012104034424
Building Laplacian operator in augmented basis...
Building 1427x928 inner product matrix...
	Time elapsed: 1.2950117588043213
Building 1427x499 inner product matrix...
	Time elapsed: 0.9069371223449707
Diagonalizing new Hermitian operator of size (1427, 1427)...
	Time elapsed: 5.263413906097412
	Time elapsed: 10.365278005599976
Filtered 244 eigenpairs.
Filtered 0 eigenpairs.
Size of composite laplacian: 1184
Projecting operator 1 onto basis...
Building 1184x928 inner product matrix...
	Time elapsed: 1.0803751945495605
Projecting operator 3 onto basis...
Building 1184x1184 inner product matrix...
	Time elapsed: 1.3328471183776855
Projecting operator 4 onto basis...
Building 1184x499 inner product matrix...
	Time elapsed: 0.7823209762573242


In [4]:
X,Y=PS.XY
x,y=PS.xy
dx=np.diff(PS.xy[0])[0]

## Tune the PhotonicSystem: substrate reflectivity and 2D material Q-factor

In [5]:
Q=2
PS.set_Beta_substrate(np.complex(M.SiO2_300nm.reflection_p(900,1/30e-7)))
PS.set_Sigma2D(0,1-1j/Q)

## Define excitation & build `EigenRasterer`

In [ ]:
DP=EP.DipoleProbe(xs=x,ys=y,tipsize=1.5) #tip size sensitively will determine magnitude of plasmon response

#--- Build Rasterer that pairs this excitation field with a Photonic2DSystem
excitation=DP(0,0) #Always get excitation field centered at the origin, which was `EigenRasterer` expects
Rasterer=EP.EigenRasterer(PS=PS,excitation=excitation)

eigenmodes2D=PS.get_eigenmodes2D(recompute=False) #This was already computed automatically by rasterer

Diagonalizing new non-Hermitian operator of size (1184, 1184)...
	Time elapsed: 2.308229923248291
Expanding right and left eigenmodes...
	Time elapsed: 16.60395312309265
Computing reflection intensity of 2D material eigenmodes...
Initializing eigenrasterer...


# Raster scan for $\lambda_P$=Rx/2

In [ ]:
#Select a plasmon wavelength
WL_scr1=Rx/2
beta0=np.complex(M.SiO2_300nm.reflection_p(900,1/30e-7))
WL1=WL_scr1/(1-beta0.real) #to give us desired screened wavelength, we have to modify the unscreened wavelength

PS.set_LambdaP(WL1)

## Examine and predict eigenmodes' "reflectivity"

In [ ]:
# --- Reflection coefficient can be computed symbolically at least for this type of Photonic2DSystem
Rs1=PS.R2D()
np.abs(Rs1).plot(color='r',label='Abs(R)')
np.imag(Rs1).plot(color='b',label='Im(R)')

plt.axvline(2*np.pi/WL_scr1,color='k',ls='--',label='intended $q_{P,scr}$')

plt.ylabel('Reflectivity')
plt.legend()

## Perform raster scan

In [ ]:
# The arguments are optional, other method is to provide photonic system as `PS=PS`,
#   and eigenmodes will be automatically (re)computed
raster1=Rasterer()

In [ ]:
np.imag(raster1).plot(cmap='bwr')
plt.gca().set_aspect('equal')
plt.clim(0.15,.42)
plt.title('$\lambda_p=%1.1f$'%WL1)

# Raster scan for $\lambda_P$=2 Rx/3

## Re-tune PhotonicSystem

In [ ]:
#Select a plasmon wavelength
WL_scr2=2*Rx/3
beta0=np.complex(M.SiO2_300nm.reflection_p(900,1/30e-7))
WL2=WL_scr2/(1-beta0.real)

PS.set_LambdaP(WL2)

In [ ]:
# --- Reflection coefficient can be computed symbolically at least for this type of Photonic2DSystem
Rs2=PS.R2D()
np.abs(Rs2).plot(color='r',label='Abs(R)')
np.imag(Rs2).plot(color='b',label='Im(R)')

plt.axvline(2*np.pi/WL_scr2,color='k',ls='--',label='intended $q_{P,scr}$')

plt.ylabel('Reflectivity')
plt.legend()

## Perform raster scan

In [ ]:
# Rasterer will automatically ask PhotonicSystem for its new eigenreflectances
# and just re-weight the ingredients to the full raster image 
raster2=Rasterer()

In [ ]:
np.imag(raster2).plot(cmap='bwr')
plt.gca().set_aspect('equal')
plt.clim(0.15,.42)
plt.title('$\lambda_p=%1.1f$'%WL2)

### Compare with conventional (brute force) raster scan

In [ ]:
#--- This does nothing clever, just projects excitation at every pixel into the PhotonicSystem
# basis and multiplies with reflection matrix, and applies the formula for generalized reflection coeff
# Calculation goes automatically in chunks to avoid projecting all the excitations at once (which would kill memory)
raster3=DP.raster_probe(PS,stride=4)

In [ ]:
np.imag(raster3).plot(cmap='bwr')
plt.gca().set_aspect('equal')
plt.clim(0.15,.42)
plt.title('$\lambda_p=%1.1f$'%WL2)